In [1]:
import sys
sys.path.append('C:\Program Files (x86)\Keysight\SD1\Libraries\Python')
sys.path.append('C:\HatCode\PXIe')
import keysightSD1
import numpy as np
import time
import h5py
import warnings
from scipy import signal
import matplotlib.pyplot as plt
import keysight_hvi as kthvi


In [ ]:
AWG = keysightSD1.SD_AOU()
AWGID = AWG.openWithSlot("", 1, 2)

In [ ]:
for i in range(4):
    AWG.AWGflush(i+1)
AWG.waveformFlush()

In [ ]:
for i in range(4):
    AWG.AWGqueueConfig(i + 1, 1)
    AWG.channelWaveShape(i + 1, keysightSD1.SD_Waveshapes.AOU_AWG)
    AWG.channelAmplitude(i + 1, 1.5)
    AWG.channelPhaseReset(i + 1)
    AWG.channelOffset(i + 1, 0)

In [ ]:
plt.plot(signal.gaussian(201, std=21))
gaussianWave = signal.gaussian(201, std=21)

In [ ]:
tWave = keysightSD1.SD_Wave()
tWave.newFromArrayDouble(0, gaussianWave)
zWave = keysightSD1.SD_Wave()
zWave.newFromArrayDouble(0, np.zeros(100))

In [ ]:
AWG.waveformLoad(tWave, 0, 0)
# waveformObject, waveformNumber, paddingMode

In [ ]:
AWG.AWGqueueWaveform(1, 0, 0, 0, 0, 0) 
# nAWG, waveformNumber, triggerMode, startDelay, cycles, prescaler

In [ ]:
AWG.channelPhaseResetMultiple('1111')
AWG.AWGstart(1)

In [ ]:
AWG.AWGstop(1)

In [ ]:
AWG.waveformFlush()
AWG.AWGflush(1)

In [ ]:
AWG.close()

In [2]:
class ApplicationConfig:
    " Defines module descriptors, configuration options and names of HVI engines, actions, triggers"
    def __init__(self):
        #Configuration options
        self.hardware_simulated = False 
        # Define options to open the instruments. Complete option list can be found in the SD1 user guide
        self.options = 'channelNumbering=keysight'
        # Define names of HVI engines, actions, triggers
        self.engine_name = "AwgEngine"
        self.awg_trigger_action_name = "AwgTrigger"
        self.fp_trigger_name = "FpTrigger"
    
    class ModuleDescriptor:
        "Descriptor for module objects"
        def __init__(self, model_number, chassis_number, slot_number, options, engine_name):
            self.model_number = model_number
            self.chassis_number = chassis_number
            self.slot_number = slot_number
            self.options = options
            self.engine_name = engine_name

    class M9031Descriptor:
        "Describes the interconnection between each pair of M9031A modules"
        def __init__(self, first_M9031_chassis_number, first_M9031_slot_number, second_M9031_chassis_number, second_M9031_slot_number):
            self.chassis_1 = first_M9031_chassis_number
            self.slot_1 = first_M9031_slot_number
            self.chassis_2 = second_M9031_chassis_number
            self.slot_2 = second_M9031_slot_number

    class Module:
        "Class defining a modular instrument object and its properties"
        def __init__(self, instrument_object, num_channels):
            self.instrument = instrument_object
            self.num_channels = num_channels

In [3]:
def open_modules(module_descriptors):
    """ 
    Opens and creates all the necessary instrument objects.
    Returns a dictionary of module objects whose keys are the HVI engine names.
    Please check SD1 3.x User Manual for options to open the instrument objects
    """
    config = ApplicationConfig()
    # Initialize output variables
    num_modules = 0
    module_dict = {} # dictionary of modules

    # Open SD1 instrument objects
    for descriptor in module_descriptors:
        if descriptor.model_number == "M3102A":
            instr_obj = keysightSD1.SD_AIN()
        else:
            instr_obj = keysightSD1.SD_AOU()
        instr_obj_options = descriptor.options + ',simulate=true' if config.hardware_simulated else descriptor.options
        id = instr_obj.openWithOptions(descriptor.model_number, descriptor.chassis_number, descriptor.slot_number, instr_obj_options)
        if id < 0:
            raise Exception("Error opening instrument in chassis: {}, slot: {}! Error code: {} - {}. Exiting...".format(descriptor.chassis_number, descriptor.slot_number, id, keysightSD1.SD_Error.getErrorMessage(id)))
        nCh = instr_obj.getOptions("channels")
        if nCh == "CH2":
            num_channels = 2
        elif nCh == "CH4":
            num_channels = 4
        else:
            raise Exception("PXI module in chassis {}, slot {} returned number of channels = {} which is incorrect. Exiting... ".format(instr_obj.getChassis(), instr_obj.getSlot(), nCh))
        module_dict[descriptor.engine_name] = config.Module(instr_obj, num_channels)
        num_modules += 1
    
    return module_dict

In [48]:
def configure_awg(awg_module, num_channels = 4):
    """ 
    awg_queue_waveform(awg_module, num_channels) queues a Gaussian waveform to all the num_channels channels of
    the AWG object awg_module that is passed to the function
    """  
    # AWG settings for all channels
    syncMode = keysightSD1.SD_SyncModes.SYNC_NONE # (SYNC_NONE / SYNC_CLK10) OR (0 / 1)
    queueMode = keysightSD1.SD_QueueMode.ONE_SHOT # (ONE_SHOT / CYCLIC)
    startDelay = 0
    prescaler = 0
    nCycles = 0
    amplitude = 1
    # Trigger settings
    triggerMode = keysightSD1.SD_TriggerModes.SWHVITRIG_CYCLE # (AUTOTRIG / SWHVITRIG / SWHVITRIG_CYCLE / EXTTRIG / EXTTRIG_CYCLE)
    
    # Load waveform to AWG memory
    awg_module.waveformFlush() #memory flush
    
    wave = keysightSD1.SD_Wave()
    wfmNum = 0
    gaussianWave = signal.gaussian(201, std=21)
    wave.newFromArrayDouble(0, gaussianWave)
    awg_module.waveformLoad(wave, wfmNum)

    Bwave = keysightSD1.SD_Wave()
    wfmNum = 1
    boxWave = np.zeros(51) + 1
    Bwave.newFromArrayDouble(0, boxWave)
    awg_module.waveformLoad(Bwave, wfmNum)
    
    for nAWG in range(1, num_channels+1):
        # AWG queue flush 
        awg_module.AWGstop(nAWG)
        awg_module.AWGflush(nAWG)
        # Set AWG mode
        awg_module.channelWaveShape(nAWG, keysightSD1.SD_Waveshapes.AOU_AWG)
        awg_module.channelAmplitude(nAWG, amplitude)
        # AWG configuration
        awg_module.AWGqueueConfig(nAWG, queueMode)
        awg_module.AWGqueueSyncMode(nAWG, syncMode)
        # Queue waveform to channel nAWG
        awg_module.AWGqueueWaveform(nAWG, wfmNum, triggerMode, startDelay, nCycles, prescaler)
        awg_module.AWGstart(nAWG) # AWG starts and wait for trigger


In [5]:
def define_hvi_resources(sys_def, module_dict, chassis_list, M9031_descriptors, pxi_sync_trigger_resources):
    """
    Configures all the necessary resources for the HVI application to execute: HW platform, engines, actions, triggers, etc. 
    """
    # Define HW platform: chassis, interconnections, PXI trigger resources, synchronization, HVI clocks
    define_hw_platform(sys_def, chassis_list, M9031_descriptors, pxi_sync_trigger_resources)

    # Define all the HVI engines to be included in the HVI
    define_hvi_engines(sys_def, module_dict)
    
    # Define list of actions to be executed
    define_hvi_actions(sys_def, module_dict)

    # Defines the trigger resources
    define_hvi_triggers(sys_def, module_dict)

In [6]:
def define_hw_platform(sys_def, chassis_list, M9031_descriptors, pxi_sync_trigger_resources):
    """
     Define HW platform: chassis, interconnections, PXI trigger resources, synchronization, HVI clocks 
    """
    config = ApplicationConfig()

    # Add chassis resources
    for chassis_number in chassis_list:
        if config.hardware_simulated:
            sys_def.chassis.add_with_options(chassis_number, 'Simulate=True,DriverSetup=model=M9018B,NoDriver=True')
        else:
            sys_def.chassis.add(chassis_number)

    # Add M9031 modules for multi-chassis setups
    if M9031_descriptors:
        interconnects = sys_def.interconnects
        for descriptor in M9031_descriptors:
            interconnects.add_M9031_modules(descriptor.chassis_1, descriptor.slot_1, descriptor.chassis_2, descriptor.slot_2)

    # Assign the defined PXI trigger resources
    sys_def.sync_resources = pxi_sync_trigger_resources
    
    # Assign clock frequencies that are outside the set of the clock frequencies of each HVI engine
    # Use the code line below if you want the application to be in sync with the 10 MHz clock
    sys_def.non_hvi_core_clocks = [10e6]


def define_hvi_engines(sys_def, module_dict):
    """
    Define all the HVI engines to be included in the HVI
    """
    # For each instrument to be used in the HVI application add its HVI Engine to the HVI Engine Collection 
    for engine_name in module_dict.keys():
        sys_def.engines.add(module_dict[engine_name].instrument.hvi.engines.main_engine, engine_name)


def define_hvi_actions(sys_def, module_dict):
    """ Defines AWG trigger actions for each module, to be executed by the "action execute" instruction in the HVI sequence
    Create a list of AWG trigger actions for each AWG module. The list depends on the number of channels """

    # Previously defined resource names
    config = ApplicationConfig()

    # For each AWG, define the list of HVI Actions to be executed and add such list to its own HVI Action Collection
    for engine_name in module_dict.keys():
        for ch_index in range(1, module_dict[engine_name].num_channels + 1):
            # Actions need to be added to the engine's action list so that they can be executed
            action_name = config.awg_trigger_action_name + str(ch_index) # arbitrary user-defined name
            instrument_action = "awg{}_trigger".format(ch_index) # name decided by instrument API
            action_id = getattr(module_dict[engine_name].instrument.hvi.actions, instrument_action)
            sys_def.engines[engine_name].actions.add(action_id, action_name)


def define_hvi_triggers(sys_def, module_dict):
    " Defines and configure the FP trigger output of each AWG "
    # Previously defined resources
    config = ApplicationConfig()
    
    # Add to the HVI Trigger Collection of each HVI Engine the FP Trigger object of that same instrument
    for engine_name in module_dict.keys():
        fp_trigger_id = module_dict[engine_name].instrument.hvi.triggers.front_panel_1
        fp_trigger = sys_def.engines[engine_name].triggers.add(fp_trigger_id, config.fp_trigger_name)
        # Configure FP trigger in each hvi.engines[index]
        fp_trigger.config.direction = kthvi.Direction.OUTPUT
        fp_trigger.config.polarity = kthvi.Polarity.ACTIVE_HIGH
        fp_trigger.config.sync_mode = kthvi.SyncMode.IMMEDIATE
        fp_trigger.config.hw_routing_delay = 0
        fp_trigger.config.trigger_mode = kthvi.TriggerMode.LEVEL 
        #NOTE: FP trigger pulse length is defined by the HVI Statements that control FP Trigger ON/OFF


In [7]:
def program_trigger_awgs(sync_block, module_dict):
    " Programs the Sync Multi-Sequence Block to trigger AWGs "
    # Recall previously defined resources
    config = ApplicationConfig()

    # Add instructions to each local sequence within the multi-sequence block
    for index in range(0, sync_block.sequences.count):
        
        # Retrieve the engine sequence from the collection of HVI Local Sequences
        # HVI Local Sequence Collection is automatically created form the user-defined HVI Engine Collection
        # Each HVI Local Sequence can be retrieved using the name alias of the corresponding HVI Engine
        sequence = sync_block.sequences[index]

        # Write FP Trigger ON to all instruments
        fp_trigger = sequence.engine.triggers[config.fp_trigger_name]
        trigger_write = sequence.instruction_set.trigger_write
        instr_trigger_ON = sequence.add_instruction("FP Trigger ON", 10, trigger_write.id)
        instr_trigger_ON.set_parameter(trigger_write.trigger, fp_trigger)
        instr_trigger_ON.set_parameter(trigger_write.sync_mode.id, trigger_write.sync_mode.IMMEDIATE)
        instr_trigger_ON.set_parameter(trigger_write.value.id, trigger_write.value.ON)

        # Write FP Trigger OFF to all instruments
        instr_trigger_OFF = sequence.add_instruction("FP Trigger OFF", 100, trigger_write.id)
        instr_trigger_OFF.set_parameter(trigger_write.trigger, fp_trigger)
        instr_trigger_OFF.set_parameter(trigger_write.sync_mode.id, trigger_write.sync_mode.IMMEDIATE)
        instr_trigger_OFF.set_parameter(trigger_write.value.id, trigger_write.value.OFF)

        # Execute AWG trigger from the HVI sequence of each module
        # "Action Execute" instruction executes the AWG trigger from HVI
        action_list = sequence.engine.actions
        instruction1 = sequence.add_instruction("AWG trigger", 10, sequence.instruction_set.action_execute.id)
        instruction1.set_parameter(sequence.instruction_set.action_execute.action.id, action_list)


In [22]:
def interact_with_hvi(hvi):
    " User-controlled HVI execution "
    config = ApplicationConfig()
    # If running with hardware, iteratively re-run HVI until the user decides to exit
    if config.hardware_simulated:
        print("Simulation completed successfully")
    else:
        while True:
            print("Press enter to run HVI again, q to exit...")
            if input() == 'q':
                break
            hvi.run(hvi.no_timeout)


In [8]:
config = ApplicationConfig()

In [9]:
module_descriptors = [config.ModuleDescriptor('M3201A', 1, 2, config.options, "0")]

In [10]:
module_dict = open_modules(module_descriptors)

In [11]:
module_dict["0"].instrument

In [49]:
for module in module_dict.values():
    configure_awg(module.instrument, module.num_channels)

In [13]:
pxi_sync_trigger_resources = [
    kthvi.TriggerResourceId.PXI_TRIGGER5, 
    kthvi.TriggerResourceId.PXI_TRIGGER6, 
    kthvi.TriggerResourceId.PXI_TRIGGER7]
chassis_list = [1]
M9031_descriptors = []

In [14]:
sys_def = kthvi.SystemDefinition("systemTest") 
# Define your system, HW platform, add HVI engines
define_hvi_resources(sys_def, module_dict, chassis_list, M9031_descriptors, pxi_sync_trigger_resources)

In [66]:
sequencer = kthvi.Sequencer("MySequencer", sys_def)

In [67]:
sync_block = sequencer.sync_sequence.add_sync_multi_sequence_block("TriggerAWGs", 30)

In [72]:
hvi = sequencer.compile()
print("HVI Compiled")
print("This HVI application needs to reserve {} PXI trigger resources to execute".format(len(hvi.compile_status.sync_resources)))


HVI Compiled
This HVI application needs to reserve 0 PXI trigger resources to execute


In [73]:
hvi.load_to_hw()    
print("HVI Loaded to HW")
hvi.run(hvi.no_timeout)
print("HVI Running...")

HVI Loaded to HW
HVI Running...


In [61]:
interact_with_hvi(hvi)

Press enter to run HVI again, q to exit...



Error: invalid usage "running not loaded HVI": HVI has to be loaded before running it

In [63]:
# Release HW resources once HVI execution is completed
hvi.release_hw()
print("Releasing HW...")

Releasing HW...


In [ ]:
# Close all modules at the end of the execution
for engine_name in module_dict:
    module_dict[engine_name].instrument.close()
print("Modules closed\n")


In [68]:
sequence = sync_block.sequences[0]

In [69]:
action_list = sequence.engine.actions
instruction1 = sequence.add_instruction("AWG trigger", 100, sequence.instruction_set.action_execute.id)

In [70]:
instruction1.set_parameter(sequence.instruction_set.action_execute.action, action_list)

In [71]:
instruction2 = sequence.add_instruction("AWG trigger 2", 300, sequence.instruction_set.action_execute.id)
instruction2.set_parameter(sequence.instruction_set.action_execute.action, action_list)

In [ ]:
" Programs the Sync Multi-Sequence Block to trigger AWGs "
# Recall previously defined resources
config = ApplicationConfig()

# Add instructions to each local sequence within the multi-sequence block
for index in range(0, sync_block.sequences.count):

    # Retrieve the engine sequence from the collection of HVI Local Sequences
    # HVI Local Sequence Collection is automatically created form the user-defined HVI Engine Collection
    # Each HVI Local Sequence can be retrieved using the name alias of the corresponding HVI Engine
    sequence = sync_block.sequences[index]

    # Write FP Trigger ON to all instruments
    fp_trigger = sequence.engine.triggers[config.fp_trigger_name]
    trigger_write = sequence.instruction_set.trigger_write
    instr_trigger_ON = sequence.add_instruction("FP Trigger ON", 10, trigger_write.id)
    instr_trigger_ON.set_parameter(trigger_write.trigger, fp_trigger)
    instr_trigger_ON.set_parameter(trigger_write.sync_mode.id, trigger_write.sync_mode.IMMEDIATE)
    instr_trigger_ON.set_parameter(trigger_write.value.id, trigger_write.value.ON)

    # Write FP Trigger OFF to all instruments
    instr_trigger_OFF = sequence.add_instruction("FP Trigger OFF", 100, trigger_write.id)
    instr_trigger_OFF.set_parameter(trigger_write.trigger, fp_trigger)
    instr_trigger_OFF.set_parameter(trigger_write.sync_mode.id, trigger_write.sync_mode.IMMEDIATE)
    instr_trigger_OFF.set_parameter(trigger_write.value.id, trigger_write.value.OFF)

    # Execute AWG trigger from the HVI sequence of each module
    # "Action Execute" instruction executes the AWG trigger from HVI
    action_list = sequence.engine.actions
    instruction1 = sequence.add_instruction("AWG trigger", 10, sequence.instruction_set.action_execute.id)
    instruction1.set_parameter(sequence.instruction_set.action_execute.action.id, action_list)
